In [2]:
import numpy as np
import pandas as pd

import torch
from model2 import *

In [18]:
df = pd.read_csv('./dataset/cityD_challengedata.csv.gz', compression='gzip')
df[:10]

,uid,d,t,x,y
0,0,0,13,133,103
1,0,0,14,143,98
2,0,0,16,162,99
3,0,0,17,149,97
4,0,0,18,144,98
5,0,0,20,160,100
6,0,0,21,155,99
7,0,0,22,143,98
8,0,0,23,163,94
9,0,0,24,146,97


In [7]:
uid_counts = df.groupby('uid').size().to_numpy()
uid_counts

array([1782, 1499, 1400, ...,  458,  963,  627])

In [29]:
def euclidean_distance(group):
    return np.sqrt(np.diff(group['x'])**2 + np.diff(group['y'])**2).sum()

dis_pu = df.groupby('uid').apply(euclidean_distance).to_numpy()
dis_pu, len(dis_pu)

AttributeError: 'function' object has no attribute 'sum'

In [22]:
for uid, traj_user in df.groupby('uid'):
    break
traj_user

,uid,d,t,x,y
0,0,0,13,133,103
1,0,0,14,143,98
2,0,0,16,162,99
3,0,0,17,149,97
4,0,0,18,144,98
...,...,...,...,...,...
1777,0,74,40,107,103
1778,0,74,41,106,102
1779,0,74,42,128,106
1780,0,74,44,115,103


In [20]:
data = {
    'uid': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
    'day': [0, 0, 1, 1, 1, 0, 1, 2, 2, 2],
    'timeslot': [10, 11, 10, 12, 13, 10, 15, 16, 10, 12],
    'x': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'y': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

df2 = pd.DataFrame(data)
distances = np.sqrt(np.diff(df1['x'])**2 + np.diff(df1['y'])**2)


In [30]:
# 示例数据
data = {
    'uid': [1, 1, 1, 2, 2, 2, 1, 1],
    'day': [0, 0, 0, 0, 0, 0, 0, 0],
    'timeslot': [0, 0, 1, 1, 1, 2, 2, 2],
    'x': [1, 2, 3, 1, 4, 5, 7, 8],
    'y': [1, 1, 2, 2, 3, 4, 4, 5]
}
df = pd.DataFrame(data)

def calculate_distances(df):
    
    # 按照uid分组
    df = df.sort_values(['uid', 'day', 'timeslot'])
    
    # 计算坐标的位移（相对于前一个点）
    df['prev_x'] = df.groupby('uid')['x'].shift(1)
    df['prev_y'] = df.groupby('uid')['y'].shift(1)
    
    # 计算欧式距离
    df['distance'] = np.sqrt((df['x'] - df['prev_x']) ** 2 + (df['y'] - df['prev_y']) ** 2)
    
    # 删除首行的NaN值
    df = df.dropna(subset=['distance'])
    
    # 返回距离列的NumPy数组
    return df['distance'].to_numpy()

# 计算距离并返回NumPy数组
distance_array = calculate_distances(df)

print(distance_array)

[1.         1.41421356 4.47213595 1.41421356 3.16227766 1.41421356]


In [7]:
model = LPBERT(4, 8, 128)

for name, param in model.named_parameters():
    print(name)

embedding_layer.day_embedding.day_embedding.weight
embedding_layer.time_embedding.time_embedding.weight
embedding_layer.location_x_embedding.location_embedding.weight
embedding_layer.location_y_embedding.location_embedding.weight
embedding_layer.timedelta_embedding.timedelta_embedding.weight
transformer_encoder.encoder_layer.self_attn.in_proj_weight
transformer_encoder.encoder_layer.self_attn.in_proj_bias
transformer_encoder.encoder_layer.self_attn.out_proj.weight
transformer_encoder.encoder_layer.self_attn.out_proj.bias
transformer_encoder.encoder_layer.linear1.weight
transformer_encoder.encoder_layer.linear1.bias
transformer_encoder.encoder_layer.linear2.weight
transformer_encoder.encoder_layer.linear2.bias
transformer_encoder.encoder_layer.norm1.weight
transformer_encoder.encoder_layer.norm1.bias
transformer_encoder.encoder_layer.norm2.weight
transformer_encoder.encoder_layer.norm2.bias
transformer_encoder.transformer_encoder.layers.0.self_attn.in_proj_weight
transformer_encoder.tra

In [4]:
traj_df = pd.read_csv('./dataset/cityB_challengedata.csv.gz', compression='gzip')

traj_df = traj_df[traj_df['uid'] < len(pd.unique(traj_df['uid'])) - 3000]
traj_df = traj_df[traj_df['uid'] >= len(pd.unique(traj_df['uid'])) * 4 / 5]

# 获取前 100 个用户的唯一 ID（按照数据出现的顺序）
first_100_uids = traj_df['uid'].unique()[:100]

# 筛选数据，保留前 100 个用户的数据
traj_df = traj_df[traj_df['uid'].isin(first_100_uids)]

traj_df

,uid,d,t,x,y
18095328,17600,0,17,79,84
18095329,17600,0,32,79,83
18095330,17600,0,33,79,84
18095331,17600,0,41,79,85
18095332,17600,1,21,78,84
...,...,...,...,...,...
18186208,17699,74,32,84,148
18186209,17699,74,33,84,149
18186210,17699,74,34,86,150
18186211,17699,74,36,85,150
